\newpage
# Modeling Task 

Short-term asset return is a challenging quantity to predict.

The objective of this task is to produce a model to predict positive moves (up trend) using a **Support Vector Machine** (SVM) model . The proposed solution should be "comprehensive with detailed feature engineering and model architecture."  

Before we start, let's remind ourselves of one of the main assumption in our quantitative finance journey: financial assets follow a Brownian motion; hence over a longer-ish period of time, we would expect our model to be predict half the time up and half the time down.  

## SVM as a classifier.  

**Support Vector Machine** is a machine learning algorithm that aims to find the hyperplane that maximizes the margins that separate either *'hardly'* or *'softly'* the data into different classes.  In this exercise, we are dealing with a binary classification (up or down).  
The hyperplane can be modelled with 
$$\theta^T \cdot X + \theta_0 = 0$$
where 

* $\theta$ is the parameters vector of the plane that maximize the margins that separate the output variables. $\theta_T$ is the transpose of that vector. 
* $\theta_0$ is a scalar vector.  How far is the hyperplane from the origin.  

In the case of *hard-margin*, we want to maximize the margin width $$\max_{\theta, \theta_0}\frac{2}{|\theta|}$$ such that there are no missclassifcation at all.  This might require to use non-linear hyperplane (*radial*, *polynomial* or *sigmoid*) or to go a higher dimension (*the Kernel Trick*) to ensure that data are then exactly separable. 
Which is the same as finding $$\min_{\theta, \theta_0} \sum_{i=1}^{N} \frac{1}{2} |\theta|^2$$

In the case of *soft-margin* where we have to weight in (with regularization) the cost of missclassification against wider margins; the loss function becomes: 
$$J(\theta) = \frac{1}{N} \sum_{i=1}^N max\left( 1-y^{(i)} \cdot (\theta^T X^{(i)} + \theta_0), 0 \right) + \lambda|\theta|^2$$

* $N$ is the number of observation in our dataset  
* $y^{(i)}$ is the binary target variable, the output
* $X^{(i)}$ is the feature vector, the inputs
* $max\left( 1-y^{(i)} \cdot (\theta^T X^{(i)} + \theta_0), 0 \right)$ is the loss function
* $\lambda|\theta|^2$ is the regularization term

## Choice of ticker 

To execute the task, we will use a mining stock **Rio Tinto Group** with ticker 'RIO'.  

As of today (22 May 2025), this is a screenshot of the stock on Yahoo Finance. 
![Screenshot of Rio](rio-screenshot.png)


## Loading Python packages, data & Initial EDA 

We'll start by loading the various Python packages required for our ML task. 
\AddToHookNext{env/Highlighting/begin}{\small}

In [ ]:
#| label: loading-packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import yfinance as yf 
import mplfinance as mpl

from datetime import date
from dateutil.relativedelta import relativedelta

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC 
from sklearn.feature_selection import SelectKBest 
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif

from sklearn.metrics import roc_curve, ConfusionMatrixDisplay, accuracy_score, auc
from sklearn.metrics import classification_report

Let's retrieve the price of Rio Tinto and store it in a *Pandas* dataframe. 
\AddToHookNext{env/Highlighting/begin}{\small}

In [ ]:
#| label: loading-data
#| output: false

# downloading the data and storing it in a dataframe
rio = yf.download('RIO')
df = pd.DataFrame(rio)

### Exploring the time-series

A quick look into the first 5 rows to check the structure.  we'll also check to see if there are missing values. 


In [ ]:
#| label: initial-eda

# get to know df variables
print('The shape of dataframe is:', df.shape)
print(df.head())

# checking for missing values. 
df.isnull().sum()

No missing values.  That's an encouraging start. 

A quick descriptive stat check on last 5 years. 


In [ ]:
#| echo: false

df.loc['2019-01-01':].describe().round(2)

### Vizualising the time-series

Initial line plot of RIO stock over the last 5 years
\AddToHookNext{env/Highlighting/begin}{\small}

In [ ]:
#| label: plot01

# Checking only over the last 5 years of data 
start_date = date.today() + relativedelta(days=int(np.round(-5*365)))
end_date = date.today()

# first vizualization 
plt.plot(df['Adj Close'].loc[start_date:end_date])
plt.show()

Using candlesticks on last 18 months of data  


In [ ]:
#| label: candlestick-plot01
#| echo: false

df_last18months = df.loc[start_date:end_date]

fig, axlist = mpl.plot(
  df_last18months, 
  type = 'candle', 
  style='binance', 
  figratio = (15, 7), 
  returnfig = True
)

fig = fig.suptitle(
  f"$RIO stock between {start_date} and {end_date}", 
  y=.95, 
  fontsize=15, 
  weight = 'semibold', 
  style = 'normal'
)

mpl.show()

##  Features Engineering 

Before starting the feature engineering process, we need to clarify what is understood by **'predicting positive moves (up trend)'.** Since *"making sense of the instructions'* is part of the assessment, we are making the following assumptions.   

  * the "moves" are based on the *closing prices* 
  * we are only trading during official market hours.  Making that assumption has some implication for trading as we would only know the closing price at the close of the trading session and hence technically we won't be able to trade that same day based on that information.  We would assume we make the trade before the closing bell  
  * question to answer: during the trading session, would tomorrow's closing price be higher than today's closing price? Both are unknown at the moment of the trade. 

With these assumptions in mind, we'll start the process of feature engineering, with the suggested features presented in the exam instructions.  

* We will create our binary classifier (*positive move*) based on the future asset price returns which we define as the log returns of the adjusted closing price.  
$$Ret_t = log \left( \frac{P_{t+1}}{P_t} \right) = log(P_{t+1}) - log(P_t)$$
* As suggested in the list of possible *'basic'* features, we can use intraday moves as signal. As prices change overtime considerably (from a minimum of adjusted close of 25.7 to 73.6), we intuitively feel, the indicator might be more relevant if we scale it as a ratio of the difference of the 2 prices over the closing price.  And to avoid look ahead biais, we define the 2 features as
$${(O-C)}_t = \frac{(O-C)_{t-1}}{C_{t-1}} \approx log \left( \frac{O_{t-1}}{C_{t-1}} \right)$$ 
$${(O-O)}_t = \frac{(O_t - O_{t-1})}{O_t} \approx log \left( \frac{O_t}{O_{t-1}} \right)$$
$${(O-prevC)}_t = \frac{(O_t - prevC_{t-1})}{O_t} \approx log \left( \frac{O_t}{C_{t-1}} \right)$$
and 
$$(H-L)_{t} = \frac{(H-L)_{t-1}}{L_{t-1}} \approx log \left( \frac{H_{t-1}}{L_{t-1}} \right)$$

* Next, we used the lagged returns: $Ret_{t-1}, Ret_{t-2}, etc.$.  Without much initial thought, we'll create 17 of these lagged returns. 

* SMA defined as $$\frac{1}{N} \sum_{i=1}^N P_{t-1}$$
* EMA defined as $$EMA_t = EMA_{t-1} + \alpha \cdot [P_t - EMA_{t-1}]$$

To do so we'll use the pandas already built-in functionalities.  We have checked in the documentation that they match the expected behavior for the exponential moving average [Pandas Doc on EMW](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.ewm.html). 

\AddToHookNext{env/Highlighting/begin}{\small}

In [ ]:
#| label: feature-eng-1

# getting returns. We shift it as it constitutes the base of what we have to predict. 
df['ret_1d_target'] = np.log(df['Adj Close'].shift(-1) / df['Adj Close'])
df['ret_1d_shifted'] = (np.log(df['Adj Close']).diff(1)).shift(1)
df.head()

# creating variables based on intra day. 
df['o-c'] = np.log(df.Open/df.Close).shift(1)   # Same day Open-Close
df['h-l'] = np.log(df.High/df.Low).shift(1)     # Same day High-Low
df['o-o'] = np.log(df.Open/df.Open.shift(1))    # Open-Previous day Open
df['o-prevC'] = np.log(df.Open/df.Close.shift(1)) # Open - Previous day Close

newcol = {}     # avoid df fragmentation warnings

# creating our lag variables
for lags in range(1, 17, 1): 
  newcol['ret_1d_lag' + str(lags) + 'd'] = df.ret_1d_shifted.shift(lags)
  newcol['o-c-lag' + str(lags) + 'd'] = df['o-c'].shift(lags)
  newcol['o-o-lag' + str(lags) + 'd'] = df['o-o'].shift(lags)
  newcol['h-l-lag' + str(lags) + 'd'] = df['h-l'].shift(lags)
  newcol['o-prevC-lag' + str(lags) + 'd'] = df['o-prevC'].shift(lags)

# Creating momentum variables
for days in range(2, 61, 3): 
  newcol['ret_'+ str(days)] = df.ret_1d_shifted.rolling(days).sum()
  newcol['sd_' + str(days)] = df.ret_1d_shifted.rolling(days).std()

# creating SMA percentage
for lags in range(2, 61, 4): 
  newcol['sma_'+str(lags)+'d']=df['Adj Close'].rolling(window=lags).mean()
  newcol['sma_'+str(lags)+'d']=np.log(df['Adj Close']/newcol['sma_'+str(lags)+'d'])
  newcol['sma_'+str(lags)+'d']=newcol['sma_'+str(lags)+'d'].shift(1)

# creating EMA percentage
for lags in range(3, 10, 2): 
  newcol['ema_'+str(lags)+'d']=df['Adj Close'].ewm(span=lags, adjust=False).mean() 
  newcol['ema_'+str(lags)+'d']=np.log(df['Adj Close']/newcol['ema_'+str(lags)+'d'])
  newcol['ema_'+str(lags)+'d']=newcol['ema_'+str(lags)+'d'].shift(1)

df_newcol = pd.DataFrame(newcol)
df = pd.concat([df, df_newcol], axis = 1)

print(df.shape)

## Target Definition and model setup 

Our initial thought to define the target variable is to use the median return over the last 5 years.  We expect this to be a slightly positive value.  The main advantage of using the median is that it ensures we have an almost perfectly balanced dataset in regards to class.  

In this sense, we define the label as 
$$y_i = \begin{cases} 1 & \text{if } x_i\geq \text{median(return)}\\ 0 & \text{otherwise} \end{cases} $$ 


### Final removing of NaN values and target definition 

\AddToHookNext{env/Highlighting/begin}{\small}

In [ ]:
#| label: create-target

# slicing the dataframe to get all data after 2019 (close to last 5 years)
df_new = df.loc['2019-01-01':].copy()
df_new.shape

# Step 2: Calculate the median of the targetcolumn in the filtered data
median_return = df_new['ret_1d_shifted'].dropna().median()
print(f"Median Return: {np.round(median_return, 6)}")

# Step 3: Create the 'target' column based on the median return
df_new['target'] = np.where(df_new['ret_1d_target'] > median_return, 1, 0)
df_new.shape
df_new['target'].value_counts(normalize = True).round(4) * 100
df_new.dropna(inplace = True)

# Step 4: Remove all unwanted columns and target variables
df3 = df_new.drop(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'target', 'ret_1d_target'], axis = 1).copy()

# checking again size of data frame
print(f"Final data set contais {df3.shape[1]} features")

# setting up our inputs and output dataframe
X = df3
y = df_new['target']

### Splitting the data 

Creating the training and testing set. Note that the training set will be use for cross-validation as well.  Testing set will only be used twice: for our initial *base model* and once we have chosen our *final model*.  

\AddToHookNext{env/Highlighting/begin}{\small}

In [ ]:
#| label: training-testing

# put shuffle to False as we are dealing with a timeseries where order of data matter.
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = False)

print('Shape of x_train:', x_train.shape)
print('Shape of y_train:', y_train.shape)
print('Shape of x_test:', x_test.shape)
print('Shape of y_test:', y_test.shape)
print(f"Size of training set is {len(x_train)} and size of testing set is {len(x_test)}")

Using 80-20 for training and testing leaves a bit over a year of data (272 trading days) for testing.  

### Checking for correlations 

Although some ML algorithms are more robust to highly correlated variables (and SVM in theory is robust to correlated variables); it is generally good practice avoiding to bring in highly correlated variables. There are several reasons for this: 

* redundancy (we prefer parsimonious model, *simpler is better* principle), 
* model interpretability (when several features are strongly correlated it becomes difficult to know which one affect the output variable), 
* model generalization. When features are strongly correlated, model can perform well on training set but generalize poorly in testing set. 
* 'curse of dimensionality'. More variables than necessary degrade the model performances. 
* stability of feature importance. With highly correlated feature, a small change in the input data can lead to significant changes in the variable importances. 

It is expected that many of our features are highly correlated (especially with the SMA and EMA).  this is because there is auto-correlation in the prices.  There are $144 X 144$ possible correlation pairs or $10296$ pairs if we remove all the self-correlated and the duplicate ones.  

Let's for instance look at the 10 most correlated (positive or negative) variables. 
\AddToHookNext{env/Highlighting/begin}{\small}

In [ ]:
#| label: correlations

corr_matrix = df3.corr()

# unstack the get the list of all pairs and put it in dataframe
corr_p = corr_matrix.unstack()      
corr_p_df = pd.DataFrame(corr_p,columns=['Correlation']).reset_index()

# Rename the columns for clarity
corr_p_df.columns = ['feature_1', 'feature_2', 'Correlation']

# Remove self-correlations
corr_p_df = corr_p_df[corr_p_df['feature_1'] != corr_p_df['feature_2']]

# Drop duplicate pairs 
corr_p_df['abs_corr'] = corr_p_df['Correlation'].abs()
corr_p_df = corr_p_df.sort_values(by='abs_corr', ascending=False).drop_duplicates(subset=['Correlation']).drop(columns=['abs_corr'])

# Sort the pairs by the absolute value of the correlation
sorted_corr_p = corr_p_df.sort_values(by='Correlation', ascending=False)

# Select the top 20 pairs
top_10_corr_pairs = sorted_corr_p.head(10)

print(f"There are {len(corr_p_df[corr_p_df['Correlation'].abs() > 0.8])} highly correlated pairs (above 0.8) from the {len(sorted_corr_p)} possible combination.")

There is clearly opportunities to reduce our amount of variables and/or reduce the correlation of our features. 


In [ ]:
#| label: correlations_table
#| echo: false

top_10_corr_pairs.iloc[0:9, ]

Now we have 2 options to build our base models.  Either, we transform the data through dimensionality reduction (PCA) to remove correlation or we select only a few 'less' correlated features.  In the next section, we will explore both cases. 

## Base model 

### Base model with PCA 

The intuition behind using PCA for our base model is 2 folds.  

* remove all form of correlation between data.  This is because all the components are orthogonal to each other in the hyperspace.  
* reduce the number of features (**parsimony principle**).  As our dataframe contains highly correlated features, we can reduce the number of features and still keep a high amount of variance in our data.  

After some trials, we decided to keep 95% of the variance in our data through the use of PCA. 
\AddToHookNext{env/Highlighting/begin}{\small}

In [ ]:
#| label: num-pca-components

#let's scale the data first 
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
print(x_train_scaled.shape)

pca = PCA()      # apply PCA with all the components
x_train_pca = pca.fit_transform(x_train_scaled)
x_train_pca.shape

cumul_var_ratio = np.cumsum(pca.explained_variance_ratio_)
print(cumul_var_ratio[1:30])

# to keep 95% of the variance 
num_comp = np.argmax(cumul_var_ratio > 0.95) + 1
print(f"{num_comp} components already explain 95% of the variance")

44 components is a lot less than the initial 144 and we are not loosing that much information. 

Now the number of components to use for a base model is itself a parameters that we can tune.  Let's see what different number of components with different kernels provide us. 
To make sure that we are not fine tuning anything with the test set in mind and also knowing that with such high number of features, we will check results on the *cross-validation training set*  using *time-series split*. 
Using a pipeline, we can now create our first base model using PCA. 

\AddToHookNext{env/Highlighting/begin}{\small}

In [ ]:
#| label: base-model-pca
#| warning: false

pca_levels = [0.8, 0.85, 0.9, 0.95, 0.99]
kernel_type = ['linear', 'rbf', 'poly', 'sigmoid']
df_xval_score = pd.DataFrame(columns = ['pca_levels','kernel_type','xval_score'])
tscv = TimeSeriesSplit(n_splits = 10, gap = 1)


for level in pca_levels: 
  for k_type in kernel_type: 
    model_svm_base = Pipeline([ 
      ('std_scaler', StandardScaler()),
      ('minmax_scaler', MinMaxScaler()), 
      ("pca", PCA(n_components = level, random_state = 42)), 
      ('classifier', SVC(kernel=k_type, probability=True, random_state=42))
      ]) 
    
    score_xval = cross_val_score(estimator = model_svm_base, 
                                 X = x_train, y = y_train, cv = tscv, 
                                 scoring = 'roc_auc', 
                                 n_jobs = 3).mean()
    new_row = pd.DataFrame({'pca_levels': [level], 
                            'kernel_type': [k_type], 
                            'xval_score': [score_xval]}) 
    df_xval_score = pd.concat([df_xval_score, new_row], ignore_index=True)
    
df_xval_score = df_xval_score.sort_values(by = 'xval_score', 
                                          ascending = False)

In [ ]:
#| label: gt_table-scores-pca
#| echo: false

df_xval_score.iloc[0:5, ]

First interesting observation is that the *rbf* kernel has not made the top of the list. Considering the non-linear nature of financial time-series, I would have intuitively thought it would be a better kernel to use. 

So 80% of the variance seems enough to get a 'decent' base model. That's equivalent to 

In [ ]:
num_comp = np.argmax(cumul_var_ratio > 0.8) + 1
print(f"{num_comp} components explain 80% of the variance")

From 144 features to 23 features.  That's a considerable reduction! 

#### Checking base model metrics 

As our model is well-balanced we can use accuracy as a metric. Not a massive difference between training set and testing set.  That's encouraging.  Results are not very encouraging just a bit over 50% accuracy.  That's expected is come from the assumption that financial assets behaved in a stochastic way (Brownian motion)

We now fit our first base model with the best PCA level and kernel. 

NOTE: empircally, we have noticed that the *MinMaxScaler()* has more impact on the cross-validation score than the *StandardScaler()*.  Also the order seems to matter: *StandardScaler()* first then *MinMaxScaler()* 

\AddToHookNext{env/Highlighting/begin}{\small}

In [ ]:
#| label: base-model-pca-fit
#| output: false

model_svm_base = Pipeline([
  ('std_scaler', StandardScaler()), 
  ('minmax_scaler', MinMaxScaler()), 
  ("pca", PCA(n_components=df_xval_score.iloc[0,0], random_state = 42)),
  ('classifier', SVC(kernel=df_xval_score.iloc[0,1], 
                     probability=True, random_state = 42))
])

model_svm_base.fit(x_train, y_train)

Let's have now a first look at the metrics.  


In [ ]:
#| label: base-model-pca-metrics

acc_train = accuracy_score(y_train, model_svm_base.predict(x_train))
acc_test = accuracy_score(y_test, model_svm_base.predict(x_test))

print(np.round(acc_train, 4))
print(np.round(acc_test, 4))

Training accuracy is quite higher than testing set indicating that we might still have quite a bit of overfitting. 

#### Confusion Matrix

Checking the confusion matrix. 

\AddToHookNext{env/Highlighting/begin}{\small}

In [ ]:
#| label: confusion_matrix-basemodel-pca

conf_mat = ConfusionMatrixDisplay.from_estimator(model_svm_base, x_train, y_train, cmap = plt.cm.Blues)
plt.title('Confusion Matrix on Base Model')
plt.show()

That's pretty informative.  The model fails mainly at identifying the up day it seems (aka poor recall).  This is maybe something we can live with (at this early stage, it seems like poor recall would be better than poor precision.)


In [ ]:
#| label: classification_report_pca

y_test_pred = model_svm_base.predict(x_test)
print(classification_report(y_test, y_test_pred))

#### ROC - AUC report 

Accuracy is an appropriate metrics when we have a balanced dataset (as in our case).  The Receiver-Operator Curve (ROC) is another metrics relevant to classification task.  It measures the True Positive Rate (TPR) also called sensitivity or recall against the False Positive Rate (FPR) (1 - sensitivity).  What is of interest is the Area Under the Curve.  An area of 0.5 being random state, anything above 0.5 shows shows increase TPR and low FPR. 

\AddToHookNext{env/Highlighting/begin}{\small}

In [ ]:
#| label: roc_auc_pca

y_prob = model_svm_base.predict_proba(x_test)[:, 1]


fpr, tpr, threshold = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
print(np.round(roc_auc, 4))

plt.clf()
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

That's not very encouraging.  This is very close to random guessing. 

### Base model with features selection 

Another way to reduce the number of available features is to use feature reduction techniques.  

There are a lots of different techniques for this.  For this report, we will demonstrate 2 ways to select features.  The first one involve the filter method and the second one involve the embedded method. 

#### The filter method 

This method is not dependent of the machine learning algorithms we are using. We are using statistical tests to determine the *k-* most valuable features. Reading at the [sklearn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest) for this method, we have the choice between 3 main statistical test: ANOVA, Chi-squared and Mutual Information.  Let's try all three on the 3 different kernels to see which one can make a good base model.  

\AddToHookNext{env/Highlighting/begin}{\small}

In [ ]:
#| label: feat-sel-kbest

stat_test_type = [chi2, f_classif, mutual_info_classif]
kbest = [10, 15, 20, 25, 30, 35]
df_xval_score = pd.DataFrame(columns = ['stat_test', 'kbest','kernel_type','xval_score'])
tscv = TimeSeriesSplit(n_splits = 10, gap = 1)

feat_scaling = Pipeline([('std_scaler', StandardScaler()), ('minmax_scaler', MinMaxScaler())])
x_train_scaled = feat_scaling.fit_transform(x_train)
x_test_scaled = feat_scaling.fit(x_test)
  
for stt in stat_test_type: 
  for kb in kbest: 
    for k_type in kernel_type: 
      skbest = SelectKBest(score_func = stt, k = kb) 
      x_train_sel = skbest.fit_transform(x_train_scaled, y_train)
      #skbest.fit(x_train_scaled, y_train) 
      #x_skb = x_train_scaled.iloc[:, skbest.get_support()]
      mod_svc = SVC(kernel = k_type, probability = True) 
      score_skb = cross_val_score(mod_svc, x_train_sel, y_train, 
                                  cv=tscv, scoring='roc_auc', n_jobs = 3).mean()
      new_row = pd.DataFrame({'stat_test': [stt.__name__], 'kbest': [kb], 
                               'kernel_type': [k_type], 'xval_score': [score_skb]})  
      df_xval_score = pd.concat([df_xval_score, new_row], ignore_index = True)
      
df_xval_score = df_xval_score.sort_values(by = 'xval_score', ascending = False)

And the top combination of kernel, k-features and stat test in cross-validation is: 

In [ ]:
#| label: gt_table-scores-kbest
#| echo: false

df_xval_score.iloc[0:5]

We can now fit our model with the chosen kernel and k-best features. Then we can check our classification report and ROC

\AddToHookNext{env/Highlighting/begin}{\small}

In [ ]:
#| label: fit_kbest_test

skbest = SelectKBest(score_func = f_classif, k = df_xval_score.iloc[0,1]) 
skbest.fit(x_train_scaled, y_train)
x_train_skb = x_train.iloc[:, skbest.get_support()]
x_test_skb  = x_test.iloc[:, skbest.get_support()]

model_svm_reduc = Pipeline([
  ('scaler', MinMaxScaler()), 
  ('minmax_scaler', MinMaxScaler()), 
  ('svm',SVC(kernel = df_xval_score.iloc[0,2], probability = True))])

mod_svc_kbest = model_svm_reduc.fit(x_train_skb, y_train)

mod_svc_kbest.fit(x_train_sel, y_train)

y_prob = mod_svc_kbest.predict_proba(x_test_skb)[:,1]


fpr, tpr, threshold = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

print(f" ROC AUC score for selected model is: {np.round(roc_auc, 4)}")

## Features engineering revisited 

Now that we have a base model with associated metrics (ROC and accuracy). We can try to be a bit more creative in getting features that we think would help us increase the *'relevance'* of our model.  

As shown in the previous section, 

## Hyper-parameters tuning 


In [ ]:
#| label: hyper-param
#| eval: false

tscv = TimeSeriesSplit(n_splits = 10, gap = 1)

model_svm_tuned = Pipeline([
  #('Scaler', MinMaxScaler()), 
  ('std_scaler', StandardScaler()), 
  ("pca", PCA(n_components = 0.95)),
  ('clf', SVC(probability = True))
])

param_grid = {'clf__C': [0.001, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 20], 
              'clf__gamma': [0.001, 0.01, 0.05, 0.1, 0.5, 1, 10, 20], 
              'clf__kernel': ['rbf', 'poly', 'linear']
}

grid = GridSearchCV(model_svm_tuned, param_grid, verbose = 1, 
                    scoring = 'roc-auc', cv = tscv, n_jobs = 3)
grid.fit(x_train, y_train)

grid.best_params_
grid.best_score_
#grid.cv_results_

## Fitting tuned model 


In [ ]:
#| eval: false

svm_best_param = SVC(**grid.best_params_)
svm_best_param.fit(x_train, y_train, 
                   #eval_set = [(x_train, y_train), (x_test, y_test)], 
                   #verbose = True
                   )
#x_val_score = cross_val_score(svm_best_param, x_train, y_train, cv = tscv)

# what does this do?  
x_train_tuned = svm_best_param.decision_function(x_train)
x_val_score 

In [ ]:
#| eval: false

y_pred = svm_best_param.predict(x_test)

acc_tuned_train = accuracy_score(y_train, svm_best_param.predict(x_train))
acc_tuned_test = accuracy_score(y_test, y_pred)

print(acc_train)
print(acc_test)

## Varibales importance 
